## Resource-Constrained Project Scheduling

This notebook demonstrates how to model and solve the classical Resource-Constrained Project Scheduling Problem 
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.


### Problem Definition

$$
\begin{aligned}
\min \quad & \max_{i \in [1..N]} \mathrm{endOf}(x_i) \qquad &\qquad & \text{(1)} \\
\text{s.t.} \quad
& \sum_{i \in [1..N]} \mathrm{pulse}(x_i, Q_{ik}) \le C_k, \qquad & \forall k \in [1..M] \quad & \text{(2)} \\
& \mathrm{endBeforeStart}(x_i, x_j), \qquad & \forall (i,j) \in P \quad & \text{(3)} \\
& \text{interval } x_i,\ \text{size} = PT_i, \qquad & \forall i \in [1..N] \quad & \text{(4)}
\end{aligned}
$$


We are given:

- A finite set of **tasks** indexed by $i \in [1..N]$
- A finite set of **renewable resources** indexed by $k \in [1..M]$

Each task $i$:
- requires a **known processing time** $PT_i > 0$,
- consumes $Q_{ik} \ge 0$ units of each resource $k$ while it runs.

Each resource $k$ has a **capacity** $C_k > 0$.

There is a set of **precedence relations** $P \subseteq [1..N]\times[1..N]$,  
where $(i,j)\in P$ means task $i$ must **finish before** task $j$ **starts**.

We decide **start times** for all tasks via interval variables $x_i$ (one per task, fixed size $PT_i$) (4).

Our goal is to build a feasible schedule such that:
- All precedence relations are respected (3).
- At any time, resource usage on each renewable resource does not exceed its capacity (2).
- The **makespan** $C_{\max}$ — the completion time of the last finishing task — is **minimized** (1).


#### Symbols and Notation

| Symbol / Function | Meaning | CPLEX CP Python Reference |
|---|---|---|
| $N$ | Number of tasks | — |
| $M$ | Number of renewable resources | — |
| $i$ | Task index ($i \in [1..N]$) | — |
| $k$ | Resource index ($k \in [1..M]$) | — |
| $PT_i$ | Processing time (duration) of task $i$ | — |
| $Q_{ik}$ | Units of resource $k$ required by task $i$ (while $i$ runs) | — |
| $C_k$ | Available capacity of resource $k$ | — |
| $x_i$ | Interval variable for task $i$ | — |
| $\mathrm{endOf}(x_i)$ | End time of task $i$ | [docplex.cp.modeler.end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\mathrm{pulse}(x_i, Q_{ik})$ | Time-varying usage of resource $k$ by $i$ while $x_i$ executes | [docplex.cp.modeler.pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\mathrm{endBeforeStart}(x_i, x_j)$ | Enforce $j$ to start after $i$ ends | [docplex.cp.modeler.end_before_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\text{interval } x_i, \text{ size} = PT_i$ | Declare $x_i$ with fixed duration | [docplex.cp.expression.interval_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html#docplex.cp.expression.interval_var) |
| $\max_{i} \mathrm{endOf}(x_i)$ | Makespan of the project | [docplex.cp.modeler.end_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min C_{\max}$ | Objective: minimize makespan | [docplex.cp.modeler.minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |


### Implementation

### Additional Resources